<a href="https://colab.research.google.com/github/aegis-iit/AEGIS/blob/master/AEGIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


ValueError: ignored

In [0]:
!ls "/content/gdrive/My Drive/"

 Aegis	 AEGIS.ipynb  'Colab Notebooks'   JAVA.zip


In [0]:
import os
os.chdir("gdrive/My Drive/Aegis")

In [0]:
!mkdir cocodataset

In [0]:
!cd cocodataset

In [0]:
!mkdir images

In [0]:
!mkdir images/train

In [0]:
!mkdir images/val

In [0]:
!mkdir images/test

In [0]:
!mkdir annotations

In [0]:
!wget http://images.cocodataset.org/zips/train2017.zip

--2019-06-20 21:06:00--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.113.139
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.113.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  47.5MB/s    in 6m 8s   

2019-06-20 21:12:08 (50.2 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]



In [0]:
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/zips/test2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget http://images.cocodataset.org/annotations/stuff_annotations_trainval2017.zip

--2019-06-20 21:13:57--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.170.219
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.170.219|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip.1’

val2017.zip.1       100%[===================>] 777.80M  43.8MB/s    in 17s     

2019-06-20 21:14:14 (45.7 MB/s) - ‘val2017.zip.1’ saved [815585330/815585330]

--2019-06-20 21:14:15--  http://images.cocodataset.org/zips/test2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.139.19
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.139.19|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6646970404 (6.2G) [application/zip]
Saving to: ‘test2017.zip.1’

test2017.zip.1      100%[===================>]   6.19G  31.9MB/s    in 3m 29s  

2019-06-20 21:17:44 (30.3 MB/s) - ‘t

In [0]:
!unzip -q val2017.zip -d images/val

[val2017.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of val2017.zip or
        val2017.zip.zip, and cannot find val2017.zip.ZIP, period.


In [0]:
!unzip -q stuff_annotations_trainval2017.zip -d annotations

In [0]:
!unzip -q train2017.zip -d images/train

In [0]:
!unzip -q test2017.zip -d images/test

In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from torchvision import datasets, transforms

from sklearn.metrics import confusion_matrix

In [0]:
USE_CUDA = True
'''
train_path = 'cocostuff/coco/images/train2017'
val_path = 'cocostuf/coco/images/val2017'
train_anno_path = 'cocostuff/coco/annotations/annotations/instances_train2017.json'
val_anno_path = 'cocostuff/coco/annotations/annotations/instances_val2017.json'
'''
#train_path = 'images/train/train2017'
val_path = 'images/val/val2017'
#train_anno_path = 'annotations/annotations/instances_train2017.json'
val_anno_path = 'annotations/annotations/instances_val2017.json'

train_path = 'images/val/val2017'
#val_path = 'images/val/val2017'
train_anno_path = 'annotations/annotations/instances_val2017.json'
#val_anno_path = 'annotations/annotations/instances_val2017.json'

#### TODO replace fully connected w/ CNN for reconstruction
#### TODO Implement regional CNN and then capsule for classification
cellID = 77
laptopID = 73
def target_t(x):
    for item in x:
        if item['category_id'] == cellID:
            return 1
    return 0
    #return x[0]['category_id']

## Change this
img_size = 64
kernel_size = 9
reduced = img_size-kernel_size + 1
reduced = int((reduced-kernel_size)/2)+1
print(reduced)

24


In [0]:
class CocoDet:
    def __init__(self, batch_size):
        ds_transform = transforms.Compose([
                       transforms.Resize((img_size,img_size)),
                       transforms.Grayscale(),
                       transforms.ToTensor()
                       #transforms.Normalize((0.1307,), (0.3081,))
                   ])

        train_dataset = datasets.CocoDetection(train_path, train_anno_path,
                                               transform=ds_transform, target_transform = target_t)
        test_dataset = datasets.CocoDetection(val_path, val_anno_path,
                                               transform=ds_transform, target_transform = target_t)
        #('../data', train=True, download=True, transform=dataset_transform)
        #test_dataset = datasets.CocoCaptions(root, annFile, transform=None, target_transform=None, transforms=None)
        #('../data', train=False, download=True, transform=dataset_transform)

        self.train_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        self.test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [0]:
class ConvLayer(nn.Module):
    def __init__(self, in_channels=1, out_channels=256, kernel_size=9):
        super(ConvLayer, self).__init__()

        self.conv = nn.Conv2d(in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=kernel_size,
                               stride=1
                             )

    def forward(self, x):
        return F.relu(self.conv(x))

In [0]:
class PrimaryCaps(nn.Module):
    def __init__(self, num_capsules=8, in_channels=256, out_channels=32, kernel_size=9):
        super(PrimaryCaps, self).__init__()

        self.capsules = nn.ModuleList([
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=2, padding=0) for _ in range(num_capsules)])

    def forward(self, x):
        u = [capsule(x) for capsule in self.capsules]
        u = torch.stack(u, dim=1)
        u = u.view(x.size(0), 32 * reduced * reduced, -1)
        return self.squash(u)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm *  input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor

In [0]:
class DigitCaps(nn.Module):
    def __init__(self, num_capsules=2, num_routes=32 * reduced * reduced, in_channels=8, out_channels=32):
        super(DigitCaps, self).__init__()

        self.in_channels = in_channels
        self.num_routes = num_routes
        self.num_capsules = num_capsules

        self.W = nn.Parameter(torch.randn(1, num_routes, num_capsules, out_channels, in_channels))

    def forward(self, x):
        batch_size = x.size(0)
        x = torch.stack([x] * self.num_capsules, dim=2).unsqueeze(4)

        W = torch.cat([self.W] * batch_size, dim=0)
        u_hat = torch.matmul(W, x)

        b_ij = Variable(torch.zeros(1, self.num_routes, self.num_capsules, 1))
        if USE_CUDA:
            b_ij = b_ij.cuda()

        num_iterations = 3
        for iteration in range(num_iterations):
            c_ij = F.softmax(b_ij)
            c_ij = torch.cat([c_ij] * batch_size, dim=0).unsqueeze(4)

            s_j = (c_ij * u_hat).sum(dim=1, keepdim=True)
            v_j = self.squash(s_j)

            if iteration < num_iterations - 1:
                a_ij = torch.matmul(u_hat.transpose(3, 4), torch.cat([v_j] * self.num_routes, dim=1))
                b_ij = b_ij + a_ij.squeeze(4).mean(dim=0, keepdim=True)

        return v_j.squeeze(1)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True) + 1e-8
        output_tensor = squared_norm *  input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor

In [0]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        '''
        self.reconstraction_layers = nn.Sequential(
            nn.Linear(16 * 2, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, img_size**2),
            #nn.ReLU(inplace=True),
            #nn.Linear(2048, img_size**2),
            nn.Sigmoid()
        )
        '''
        #img_size was 64 initially
        self.linear = torch.nn.Linear(64, 1024*4*4)

        self.conv1 = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=1024, out_channels=512, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=512, out_channels=256, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=256, out_channels=128, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )

        self.conv4 = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=128, out_channels=1, kernel_size=4,
                stride=2, padding=1, bias=False
            )
        )

        self.out = torch.nn.Tanh()

    def forward(self, x, data):
        classes = torch.sqrt((x ** 2).sum(2))
        classes = F.softmax(classes)

        _, max_length_indices = classes.max(dim=1)
        masked = Variable(torch.sparse.torch.eye(2))
        if USE_CUDA:
            masked = masked.cuda()
        masked = masked.index_select(dim=0, index=max_length_indices.squeeze(1).data)

        #reconstructions = self.reconstraction_layers((x * masked[:, :, None, None]).view(x.size(0), -1))
        #reconstructions = reconstructions.view(-1, 1, img_size, img_size)
        #multiplication by mask = way of conditioning reconstruction ?
        rec = self.linear((x * masked[:, :, None, None]).view(x.size(0), -1))
        rec = rec.view(rec.shape[0], 1024, 4, 4)
        rec = self.conv1(rec)
        rec = self.conv2(rec)
        rec = self.conv3(rec)
        rec = self.conv4(rec)
        rec= self.out(rec)
        #print(x.size())
        #print(data.size())
        return rec, masked
#        return reconstructions, masked

In [0]:
class CapsNet(nn.Module):
    def __init__(self):
        super(CapsNet, self).__init__()
        self.conv_layer = ConvLayer()
        self.primary_capsules = PrimaryCaps()
        self.digit_capsules = DigitCaps()
        self.decoder = Decoder()

        self.mse_loss = nn.MSELoss()

    def forward(self, data):
        output = self.digit_capsules(self.primary_capsules(self.conv_layer(data)))
        #print(output.sum())
        reconstructions, masked = self.decoder(output, data)
        return output, reconstructions, masked

    def loss(self, data, x, target, reconstructions):
        #print('ml: ', self.margin_loss(x, target))
        #print('rec: ', self.reconstruction_loss(data, reconstructions))
        return self.margin_loss(x, target) + self.reconstruction_loss(data, reconstructions)

    def margin_loss(self, x, labels, size_average=True):
        batch_size = x.size(0)

        #norm of x = probability of being each class
        v_c = torch.sqrt((x**2).sum(dim=2, keepdim=True))

        #if prediction is correct at proba 0.9 or above
        left = F.relu(0.9 - v_c).view(batch_size, -1)
        #if prediction is incorrect w/ proba 0.1 or above
        right = F.relu(v_c - 0.1).view(batch_size, -1)

        # loss = left side if capsule corresponds to pred, right ow
        loss = labels * left + 0.5 * (1.0 - labels) * right
        # mean of loss on the batch
        loss = loss.sum(dim=1).mean()
        #print('ml: ', loss)

        return loss

    def reconstruction_loss(self, data, reconstructions):
        loss = self.mse_loss(reconstructions.view(reconstructions.size(0), -1),
                             data.view(reconstructions.size(0), -1))
        #print 'rec: ', loss * 0.0005
        return loss * 0.0005

In [0]:
capsule_net = CapsNet()
if USE_CUDA:
    capsule_net = capsule_net.cuda()
optimizer = Adam(capsule_net.parameters(), lr=1e-3)

n_epochs = 10
batch_size = 32#128
eps = 1e-7

cocod = CocoDet(batch_size)



total = len(cocod.train_loader)
print('total_obs: ', total*batch_size)

print('bpe: ', total)

loading annotations into memory...
Done (t=1.78s)
creating index...
index created!
loading annotations into memory...
Done (t=0.63s)
creating index...
index created!
total_obs:  5024
bpe:  157


In [0]:
for epoch in range(n_epochs):
    capsule_net.train()
    train_loss = 0
    print(epoch)
    for batch_id, (data, target) in enumerate(cocod.train_loader):
        #print(data.type())
        target = torch.sparse.torch.eye(2).index_select(dim=0, index=target)
        data, target = Variable(data.float()), Variable(target.float())
        #print(data.type())
        if USE_CUDA:
            data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output, reconstructions, masked = capsule_net(data)
        loss = capsule_net.loss(data, output, target, reconstructions)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()#loss.data[0]

        if batch_id % 3 == 0:
            y_pred = np.argmax(masked.data.cpu().numpy(), 1)
            y_true = np.argmax(target.data.cpu().numpy(), 1)
            cm = confusion_matrix(y_true,y_pred)
            tn, fp, fn, tp = cm.ravel()
            print(cm)

            print("train accuracy:", sum(np.argmax(masked.data.cpu().numpy(), 1) ==
                                   np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size))
            print("train precision:", tp/(tp+fp+eps))
            print("train recall:", tp/(tp+fn+eps))
            #print(output.sum())

    print(train_loss / len(cocod.train_loader))
    #print( np.argmax(masked.data.cpu().numpy()))

    capsule_net.eval()
    test_loss = 0
    '''
    for batch_id, (data, target) in enumerate(cocod.test_loader):

        target = torch.sparse.torch.eye(10).index_select(dim=0, index=target)
        data, target = Variable(data), Variable(target)

        if USE_CUDA:
            data, target = data.cuda(), target.cuda()

        output, reconstructions, masked = capsule_net(data)
        loss = capsule_net.loss(data, output, target, reconstructions)

        test_loss += loss.item()#loss.data[0]

        if batch_id % 10 == 0:
            print("test accuracy:", sum(np.argmax(masked.data.cpu().numpy(), 1) ==
                                   np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size))
                 

    print(test_loss / len(mnist.test_loader))
    '''

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[[11 19]
 [ 1  1]]
train accuracy: 0.375
train precision: 0.049999999749999996
train recall: 0.4999999750000013
[[23  9]
 [ 0  0]]
train accuracy: 0.71875
train precision: 0.0
train recall: 0.0
[[16 15]
 [ 0  1]]
train accuracy: 0.53125
train precision: 0.062499999609374995
train recall: 0.9999999000000099
[[19 10]
 [ 1  2]]
train accuracy: 0.65625
train precision: 0.1666666652777778
train recall: 0.6666666444444452
[[20 10]
 [ 1  1]]
train accuracy: 0.65625
train precision: 0.09090909008264464
train recall: 0.4999999750000013
[[16 11]
 [ 3  2]]
train accuracy: 0.5625
train precision: 0.15384615266272192
train recall: 0.39999999200000014
[[19 12]
 [ 1  0]]
train accuracy: 0.59375
train precision: 0.0
train recall: 0.0
[[20 11]
 [ 0  1]]
train accuracy: 0.65625
train precision: 0.0833333326388889
train recall: 0.9999999000000099
[[20 11]
 [ 1  0]]
train accuracy: 0.625
train precision: 0.0
train recall: 0.0
[[17 13]
 [ 2  0]]
train accuracy: 0.53125
train precision: 0.0
train recall: 0.